## Research Ethics
This code makes use of the mitdeeplearning package (Amini, 2024) for the data loading function. 
The DB-VAE for Models 4-6 is inspired by the Debiasing Computer Vision Lab notebook from 6.S191.

### Copyright 2024 MIT 6.S191 Introduction to Deep Learning. All Rights Reserved. 
 
Licensed under the MIT License. You may not use this file except in compliance 
with the License. Use and/or modification of this code outside of 6.S191 must 
reference: 

© MIT 6.S191: Introduction to Deep Learning 
http://introtodeeplearning.com 

In [ ]:
### Imports & Comet Setup ###
COMET_API_KEY = # Insert KEY here
import comet_ml

### Create a Comet experiment to track our training run ###
def create_experiment(project_name, params):
    # end any prior experiments
    if 'experiment' in locals():
        experiment.end()

    # initiate the comet experiment for tracking
    experiment = comet_ml.Experiment(
    api_key=COMET_API_KEY,
    project_name=project_name)
    # log our hyperparameters, defined above, to the experiment
    for param, value in params.items():
        experiment.log_parameter(param, value)
    experiment.flush()

    return experiment

import os
CWD = os.getcwd()
print(CWD)

### Data Visualization ###
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['font.family'] = "Times New Roman"

### Data Loading ###
import h5py
import mitdeeplearning as mdl

### The Rest ###
import numpy as np 
import pandas as pd 
import tensorflow as tf 
from tensorflow import keras
from keras import backend, layers, utils
import functools

assert len(tf.config.list_physical_devices('GPU')) > 0
assert COMET_API_KEY != "", "Please insert your Comet API Key"
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)

# Create our Standard ResNet50V2 CNN

In [ ]:
### Standard CNN ###

# Helper Functions
def resize_images(x):
    return tf.image.resize(x, (64, 64))

# CNN Function
def make_standard_ResNet50_V2(n_outputs = 1):
    
    Resize = tf.keras.layers.Lambda(resize_images)
    Flatten = tf.keras.layers.Flatten
    Dense = functools.partial(tf.keras.layers.Dense, activation='relu')
    ResNet50V2 = tf.keras.applications.ResNet50V2(
        include_top=False,
        weights="imagenet", # Utilizing Transfer Learning, also maintains consistency
        input_tensor=None,
        input_shape=(64,64,3),
        pooling=None,
        classes=1000,
        classifier_activation="softmax",
    )
    ResNet50V2 = tf.keras.Model(inputs = ResNet50V2.layers[1].input, 
                                outputs = ResNet50V2.layers[-1].output)

    model = tf.keras.Sequential()
    
    model.add(Resize)
    model.add(ResNet50V2)
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Dense(n_outputs, activation=None))

    return model

# Create our DB-VAE

In [ ]:
### Define Decoder Network ###
latent_dim = 100 # number of latent variables
n_filters = 12 
def make_decoder_network():
    """
    Layer Types, Functional Definition
    """
    Conv2DTranspose = functools.partial(tf.keras.layers.Conv2DTranspose, padding='same', activation='relu')
    Dense = functools.partial(tf.keras.layers.Dense, activation='relu')
    Reshape = tf.keras.layers.Reshape 
    BatchNormalization = tf.keras.layers.BatchNormalization
    LeakyReLU = tf.keras.layers.LeakyReLU
    # Decoder
    decoder = tf.keras.Sequential([
        Dense(units=4*4*6*n_filters),
        Reshape(target_shape=(4,4,6*n_filters)),

        Conv2DTranspose(256, (4, 4), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),

        Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),

        Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),

        Conv2DTranspose(3, (4, 4), strides=(2, 2), padding='same', activation='sigmoid')
    ])
    
    return decoder

In [ ]:
### DB_VAE Helper Functions ###

### VAE Reparameterization ###
def sampling(z_mean, z_logsigma):
    batch, latent_dim = z_mean.shape
    epsilon = tf.random.normal(shape=(batch, latent_dim))
    z = z_mean + tf.math.exp(0.5 * z_logsigma) * epsilon
    return z

### Defining the VAE loss function ###
def vae_loss_function(x, x_recon, mu, logsigma, kl_weight=0.0005):
  latent_loss = 0.5 * tf.reduce_sum(tf.exp(logsigma) + tf.square(mu) - 1.0 - logsigma, axis=1)
  reconstruction_loss = tf.reduce_mean(tf.abs(x-x_recon), axis=(1,2,3))
  vae_loss = kl_weight * latent_loss + reconstruction_loss
  return vae_loss

### Loss function for DB-VAE ###
def debiasing_loss_function(x, x_pred, y, y_logit, mu, logsigma):
  vae_loss = vae_loss_function(x, x_pred, mu, logsigma)
  classification_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=y_logit)
  malignance_indicator = tf.cast(tf.equal(y, 1), tf.float32)
  total_loss = tf.reduce_mean(
      classification_loss +
      malignance_indicator * vae_loss
  )
  return total_loss, classification_loss

In [ ]:
### Defining and creating the DB-VAE ###

class DB_VAE(tf.keras.Model):
  def __init__(self, latent_dim):
    super(DB_VAE, self).__init__()
    self.latent_dim = latent_dim

    # Define the number of outputs for the encoder. Recall that we have
    # `latent_dim` latent variables, as well as a supervised output for the
    # classification.
    num_encoder_dims = 2*self.latent_dim + 1

    self.encoder = make_standard_ResNet50_V2(num_encoder_dims)
    self.decoder = make_decoder_network()

  def encode(self, x):
    encoder_output = self.encoder(x)
    y_logit = tf.expand_dims(encoder_output[:, 0], -1)
    z_mean = encoder_output[:, 1:self.latent_dim+1]
    z_logsigma = encoder_output[:, self.latent_dim+1:]

    return y_logit, z_mean, z_logsigma

  def reparameterize(self, z_mean, z_logsigma):
    z = sampling(z_mean, z_logsigma)
    return z

  def decode(self, z):
    reconstruction = self.decoder(z)
    return reconstruction

  def call(self, x):
    y_logit, z_mean, z_logsigma = self.encode(x)
    z = self.reparameterize(z_mean, z_logsigma)
    recon = self.decode(z)
    return y_logit, z_mean, z_logsigma, recon

  def predict(self, x):
    y_logit, z_mean, z_logsigma = self.encode(x)
    return y_logit

dbvae = DB_VAE(latent_dim)

## ISIC Data Loading

In [ ]:
### Examine Dataset ###
with h5py.File(f'{CWD}/datasets/ISIC/ISIC.h5','r') as f:
    # Print the keys (names) of all groups and datasets in the file
    print("Keys:", list(f.keys()))

    # Iterate through each key and print more detailed information
    for key in f.keys():
        if isinstance(f[key], h5py.Dataset):
            print(f"Dataset: {key}")
            print("  Shape:", f[key].shape)
            print("  Data type:", f[key].dtype)

### Instantiate Loader Function ###
path_to_training_data= f'{CWD}/datasets/ISIC/ISIC.h5'
loader_ISIC = mdl.lab2.TrainingDatasetLoader(path_to_training_data)

### Visualize our data ###
number_of_training_examples = loader_ISIC.get_train_size()
print(number_of_training_examples)
(images, labels) = loader_ISIC.get_batch(100)
malignant_images = images[np.where(labels==1)[0]]
benign_images = images[np.where(labels==0)[0]]

idx_malignant = 23
idx_benign = 9

plt.figure(figsize=(5,5))
plt.subplot(1, 2, 1)
plt.imshow(malignant_images[idx_malignant])
plt.title("Malignant"); plt.grid(False)

plt.subplot(1, 2, 2)
plt.imshow(benign_images[idx_benign])
plt.title("Benign"); plt.grid(False)

## ISIC_DiDI Data Loading

In [ ]:
### Examine Dataset ###
with h5py.File(f'{CWD}/datasets/ISIC_DiDI/ISIC_DiDI.h5','r') as f:
    # Print the keys (names) of all groups and datasets in the file
    print("Keys:", list(f.keys()))

    # Iterate through each key and print more detailed information
    for key in f.keys():
        if isinstance(f[key], h5py.Dataset):
            print(f"Dataset: {key}")
            print("  Shape:", f[key].shape)
            print("  Data type:", f[key].dtype)

### Instantiate Loader Function ###
path_to_training_data= f'{CWD}/datasets/ISIC_DiDI/ISIC_DiDI.h5'
loader_ISIC_DiDI = mdl.lab2.TrainingDatasetLoader(path_to_training_data)

### Visualize our data ###
number_of_training_examples = loader_ISIC_DiDI.get_train_size()
print(number_of_training_examples)
(images, labels) = loader_ISIC_DiDI.get_batch(100)
malignant_images = images[np.where(labels==1)[0]]
benign_images = images[np.where(labels==0)[0]]

idx_malignant = 23
idx_benign = 9

plt.figure(figsize=(5,5))
plt.subplot(1, 2, 1)
plt.imshow(malignant_images[idx_malignant])
plt.title("Malignant"); plt.grid(False)

plt.subplot(1, 2, 2)
plt.imshow(benign_images[idx_benign])
plt.title("Benign"); plt.grid(False)

## ISIC_ArGI Data Loading

In [ ]:
### Examine Dataset ###
with h5py.File(f'{CWD}/datasets/ISIC_ArGI/ISIC_ArGI.h5','r') as f:
    # Print the keys (names) of all groups and datasets in the file
    print("Keys:", list(f.keys()))

    # Iterate through each key and print more detailed information
    for key in f.keys():
        if isinstance(f[key], h5py.Dataset):
            print(f"Dataset: {key}")
            print("  Shape:", f[key].shape)
            print("  Data type:", f[key].dtype)

### Instantiate Loader Function ###
path_to_training_data= f'{CWD}/datasets/ISIC_ArGI/ISIC_ArGI.h5'
loader_ISIC_ArGI = mdl.lab2.TrainingDatasetLoader(path_to_training_data)

### Visualize our data ###
number_of_training_examples = loader_ISIC_DiDI.get_train_size()
print(number_of_training_examples)
(images, labels) = loader_ISIC_ArGI.get_batch(100)
malignant_images = images[np.where(labels==1)[0]]
benign_images = images[np.where(labels==0)[0]]

idx_malignant = 23
idx_benign = 9

plt.figure(figsize=(5,5))
plt.subplot(1, 2, 1)
plt.imshow(malignant_images[idx_malignant])
plt.title("Malignant"); plt.grid(False)

plt.subplot(1, 2, 2)
plt.imshow(benign_images[idx_benign])
plt.title("Benign"); plt.grid(False)

# Model 1 - Standard ResNet50V2, trained on ISIC alone.

In [ ]:
model_1 = make_standard_ResNet50_V2()

In [ ]:
### Train the standard CNN ###

# Training hyperparameters
params = dict(
  batch_size = 32,
  num_epochs = 50,  # keep small to run faster
  learning_rate = 5e-4,
)

experiment = create_experiment("Model_1", params)

optimizer = tf.keras.optimizers.Adam(params["learning_rate"]) # define our optimizer
loss_history = mdl.util.LossHistory(smoothing_factor=0.99) # to record loss evolution
plotter = mdl.util.PeriodicPlotter(sec=2, scale='semilogy')
if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists

@tf.function
def standard_train_step(x, y):
  with tf.GradientTape() as tape:
    # feed the images into the model
    logits = model_1(x)
    # Compute the loss
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)

  # Backpropagation
  grads = tape.gradient(loss, model_1.trainable_variables)
  optimizer.apply_gradients(zip(grads, model_1.trainable_variables))
  return loss

# The training loop!
step = 0
for epoch in range(params["num_epochs"]):
  for idx in tqdm(range(loader_ISIC.get_train_size()//params["batch_size"])):
    # Grab a batch of training data and propagate through the network
    x, y = loader_ISIC.get_batch(params["batch_size"])

    loss = standard_train_step(x, y)

    # Record the loss and plot the evolution of the loss as a function of training
    loss_history.append(loss.numpy().mean())
    plotter.plot(loss_history.get())

    experiment.log_metric("loss", loss.numpy().mean(), step=step)
    step += 1

In [ ]:
### Obtain Loss Values Over Epoch ###
steps = len(loss_history.get())
print(steps)

epochs = np.uint8(params['num_epochs'])
batches_per_epoch = np.uint8(steps/epochs)

loss_hist = np.zeros((steps,1))
for i in range(steps):
    loss_hist[i] = loss_history.get()[i]
loss_hist = loss_hist.reshape(epochs, batches_per_epoch)

row_means = np.zeros((epochs))
for i in range(epochs):
    row_means[i] = loss_hist.sum(axis=1)[i]

row_means = row_means/batches_per_epoch
for mean in row_means:
    print(mean)

In [ ]:
# Graph Loss
import matplotlib
matplotlib.rcParams['font.family'] = "Times New Roman"
epochs_range = np.arange(1,51)
plt.figure(figsize=(8,7))
size_axis_titles = 16
size_title = 18
size_legend = 14
plt.xlabel("Epoch", fontsize=size_axis_titles)
plt.ylabel("Loss", fontsize = size_axis_titles)
plt.axis([1, 50, 0, 2.5])
plt.plot(epochs_range,row_means,label='Training')
plt.legend(loc='upper right', fontsize=size_legend)
plt.title(f'Training Loss for Model 1', fontsize=size_title)
plt.show()

In [ ]:
### Evaluation of standard CNN on ISIC ###
n=30
accuracies = np.zeros((n,1))
for i in range(n):
    (batch_x, batch_y) = loader_ISIC.get_batch(256) # Matches batch size
    y_pred_standard = tf.round(tf.nn.sigmoid(model_1.predict(batch_x)))
    acc_standard = tf.reduce_mean(tf.cast(tf.equal(batch_y, y_pred_standard), tf.float32))
    accuracies[i] = acc_standard.numpy()

print(accuracies.mean())
print(accuracies.std())

In [ ]:
### Evaluation of standard CNN on ISIC_DiDI ###
n=30
accuracies = np.zeros((n,1))

for i in range(n):
    (batch_x, batch_y) = loader_ISIC_DiDI.get_batch(256) # Matches batch size
    y_pred_standard = tf.round(tf.nn.sigmoid(model_1.predict(batch_x)))
    acc_standard = tf.reduce_mean(tf.cast(tf.equal(batch_y, y_pred_standard), tf.float32))
    accuracies[i] = acc_standard.numpy()

print(accuracies.mean())
print(accuracies.std())

In [ ]:
### Clear Previous Session ###
tf.keras.backend.clear_session()
experiment.end()

# Model 2 - Standard ResNet50V2, trained on ISIC_DiDI.

In [ ]:
model_2 = make_standard_ResNet50_V2()

In [ ]:
### Train the standard CNN ###

# Training hyperparameters
params = dict(
  batch_size = 32,
  num_epochs = 50,  # keep small to run faster
  learning_rate = 5e-4,
)

experiment = create_experiment("Model_2", params)

optimizer = tf.keras.optimizers.Adam(params["learning_rate"]) # define our optimizer
loss_history = mdl.util.LossHistory(smoothing_factor=0.99) # to record loss evolution
plotter = mdl.util.PeriodicPlotter(sec=2, scale='semilogy')
if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists

@tf.function
def standard_train_step(x, y):
  with tf.GradientTape() as tape:
    # feed the images into the model
    logits = model_2(x)
    # Compute the loss
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)

  # Backpropagation
  grads = tape.gradient(loss, model_2.trainable_variables)
  optimizer.apply_gradients(zip(grads, model_2.trainable_variables))
  return loss

# The training loop!
step = 0
for epoch in range(params["num_epochs"]):
  for idx in tqdm(range(loader_ISIC_DiDI.get_train_size()//params["batch_size"])):
    # Grab a batch of training data and propagate through the network
    x, y = loader_ISIC_DiDI.get_batch(params["batch_size"])

    loss = standard_train_step(x, y)

    # Record the loss and plot the evolution of the loss as a function of training
    loss_history.append(loss.numpy().mean())
    plotter.plot(loss_history.get())

    experiment.log_metric("loss", loss.numpy().mean(), step=step)
    step += 1

In [ ]:
### Obtain Loss Values Over Epoch ###
steps = len(loss_history.get())
print(steps)

epochs = np.uint8(params['num_epochs'])
batches_per_epoch = np.uint8(steps/epochs)

loss_hist = np.zeros((steps,1))
for i in range(steps):
    loss_hist[i] = loss_history.get()[i]
loss_hist = loss_hist.reshape(epochs, batches_per_epoch)

row_means = np.zeros((epochs))
for i in range(epochs):
    row_means[i] = loss_hist.sum(axis=1)[i]

row_means = row_means/batches_per_epoch
for mean in row_means:
    print(mean)

In [ ]:
### Graph Loss ###
import matplotlib
matplotlib.rcParams['font.family'] = "Times New Roman"
epochs_range = np.arange(1,51)
plt.figure(figsize=(8,7))
size_axis_titles = 16
size_title = 18
size_legend = 14
plt.xlabel("Epoch", fontsize=size_axis_titles)
plt.ylabel("Loss", fontsize = size_axis_titles)
plt.axis([1, 50, 0, 2.5])
plt.plot(epochs_range,row_means,label='Training')
plt.legend(loc='upper right', fontsize=size_legend)
plt.title(f'Training Loss for Model 2', fontsize=size_title)
plt.show()

In [ ]:
### Evaluation of standard CNN on ISIC ###
n=30
accuracies = np.zeros((n,1))

for i in range(n):
    (batch_x, batch_y) = loader_ISIC.get_batch(256) # Matches batch size
    y_pred_standard = tf.round(tf.nn.sigmoid(model_2.predict(batch_x)))
    acc_standard = tf.reduce_mean(tf.cast(tf.equal(batch_y, y_pred_standard), tf.float32))
    accuracies[i] = acc_standard.numpy()

print(accuracies.mean())
print(accuracies.std())

In [ ]:
### Evaluation of standard CNN on ISIC_DiDI ###
n=30
accuracies = np.zeros((n,1))

for i in range(n):
    (batch_x, batch_y) = loader_ISIC_DiDI.get_batch(256) # Matches batch size
    y_pred_standard = tf.round(tf.nn.sigmoid(model_2.predict(batch_x)))
    acc_standard = tf.reduce_mean(tf.cast(tf.equal(batch_y, y_pred_standard), tf.float32))
    accuracies[i] = acc_standard.numpy()

print(accuracies.mean())
print(accuracies.std())

In [ ]:
### Clear Previous Session ###
tf.keras.backend.clear_session()
experiment.end()

# Model 3

In [ ]:
model_3 = make_standard_ResNet50_V2()

In [ ]:
### Train the standard CNN ###

# Training hyperparameters
params = dict(
  batch_size = 32,
  num_epochs = 50,  # keep small to run faster
  learning_rate = 5e-4,
)

experiment = create_experiment("Model_3", params)

optimizer = tf.keras.optimizers.Adam(params["learning_rate"]) # define our optimizer
loss_history = mdl.util.LossHistory(smoothing_factor=0.99) # to record loss evolution
plotter = mdl.util.PeriodicPlotter(sec=2, scale='semilogy')
if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists

@tf.function
def standard_train_step(x, y):
  with tf.GradientTape() as tape:
    # feed the images into the model
    logits = model_3(x)
    # Compute the loss
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)

  # Backpropagation
  grads = tape.gradient(loss, model_3.trainable_variables)
  optimizer.apply_gradients(zip(grads, model_3.trainable_variables))
  return loss

# The training loop!
step = 0
for epoch in range(params["num_epochs"]):
  for idx in tqdm(range(loader_ISIC_ArGI.get_train_size()//params["batch_size"])):
    # Grab a batch of training data and propagate through the network
    x, y = loader_ISIC_ArGI.get_batch(params["batch_size"])

    loss = standard_train_step(x, y)

    # Record the loss and plot the evolution of the loss as a function of training
    loss_history.append(loss.numpy().mean())
    plotter.plot(loss_history.get())

    experiment.log_metric("loss", loss.numpy().mean(), step=step)
    step += 1

In [ ]:
### Obtain Loss Values Over Epoch ###
steps = len(loss_history.get())
print(steps)

epochs = np.uint8(params['num_epochs'])
batches_per_epoch = np.uint8(steps/epochs)

loss_hist = np.zeros((steps,1))
for i in range(steps):
    loss_hist[i] = loss_history.get()[i]
loss_hist = loss_hist.reshape(epochs, batches_per_epoch)

row_means = np.zeros((epochs))
for i in range(epochs):
    row_means[i] = loss_hist.sum(axis=1)[i]

row_means = row_means/batches_per_epoch
for mean in row_means:
    print(mean)

In [ ]:
### Graph Loss ###
import matplotlib
matplotlib.rcParams['font.family'] = "Times New Roman"
epochs_range = np.arange(1,51)
plt.figure(figsize=(8,7))
size_axis_titles = 16
size_title = 18
size_legend = 14
plt.xlabel("Epoch", fontsize=size_axis_titles)
plt.ylabel("Loss", fontsize = size_axis_titles)
plt.axis([1, 50, 0, 2.5])
plt.plot(epochs_range,row_means,label='Training')
plt.legend(loc='upper right', fontsize=size_legend)
plt.title(f'Training Loss for Model 3', fontsize=size_title)
plt.show()

In [ ]:
### Evaluation of standard CNN on ISIC ###
n=30
accuracies = np.zeros((n,1))

for i in range(n):
    (batch_x, batch_y) = loader_ISIC.get_batch(256) # Matches batch size
    y_pred_standard = tf.round(tf.nn.sigmoid(model_3.predict(batch_x)))
    acc_standard = tf.reduce_mean(tf.cast(tf.equal(batch_y, y_pred_standard), tf.float32))
    accuracies[i] = acc_standard.numpy()

print(accuracies.mean())
print(accuracies.std())

In [ ]:
### Evaluation of standard CNN on ISIC_DiDI ###
n=30
accuracies = np.zeros((n,1))

for i in range(n):
    (batch_x, batch_y) = loader_ISIC_DiDI.get_batch(256) # Matches batch size
    y_pred_standard = tf.round(tf.nn.sigmoid(model_3.predict(batch_x)))
    acc_standard = tf.reduce_mean(tf.cast(tf.equal(batch_y, y_pred_standard), tf.float32))
    accuracies[i] = acc_standard.numpy()

print(accuracies.mean())
print(accuracies.std())

# Models 4, 5, and 6
Between training different models, make sure to rerun the initialization code below to reset the weights of the dbvae.


In [ ]:
dbvae = DB_VAE(latent_dim)

In [ ]:
### DB_VAE Training Helper Functions ###

# Function to return the means for an input image batch
def get_latent_mu(images, dbvae, batch_size=1024):
    N = images.shape[0]
    mu = np.zeros((N, latent_dim))
    for start_ind in range(0, N, batch_size):
        end_ind = min(start_ind+batch_size, N+1)
        batch = (images[start_ind:end_ind]).astype(np.float32)/255.
        _, batch_mu, _ = dbvae.encode(batch)
        mu[start_ind:end_ind] = batch_mu
    return mu

def get_training_sample_probabilities(images, dbvae, bins=10, smoothing_fac=0.001):
    print("Recomputing the sampling probabilities")
    mu = get_latent_mu(images, dbvae)
    training_sample_p = np.zeros(mu.shape[0])
    for i in range(latent_dim):
        latent_distribution = mu[:,i]
        hist_density, bin_edges =  np.histogram(latent_distribution, density=True, bins=bins)
        bin_edges[0] = -float('inf')
        bin_edges[-1] = float('inf')
        bin_idx = np.digitize(latent_distribution, bin_edges)
        hist_smoothed_density = hist_density + smoothing_fac
        hist_smoothed_density = hist_smoothed_density / np.sum(hist_smoothed_density)
        p = 1.0/(hist_smoothed_density[bin_idx-1])
        p = p / np.sum(p)
        training_sample_p = np.maximum(p, training_sample_p)
    training_sample_p /= np.sum(training_sample_p)

    return training_sample_p

In [ ]:
### Training the DB-VAE on ISIC ###
import IPython
# Hyperparameters
params = dict(
  batch_size = 32,
  learning_rate = 5e-4,
  latent_dim = 100,
  num_epochs = 50, #DB-VAE needs slightly more epochs to train
)

experiment = create_experiment("Model_4", params)

# instantiate a new DB-VAE model and optimizer
dbvae = DB_VAE(params["latent_dim"])
optimizer = tf.keras.optimizers.Adam(params["learning_rate"])

@tf.function
def debiasing_train_step(x, y):

  with tf.GradientTape() as tape:
    y_logit, z_mean, z_logsigma, x_recon = dbvae(x)
    loss, class_loss = debiasing_loss_function(x, x_recon, y, y_logit, z_mean, z_logsigma)
  grads = tape.gradient(loss, dbvae.trainable_variables)
  optimizer.apply_gradients(zip(grads, dbvae.trainable_variables))
  return loss

all_imgs = loader_ISIC.get_all_train_faces()

if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists

loss_history_2 = mdl.util.LossHistory(smoothing_factor=0.99) # to record loss evolution
# The training loop -- outer loop iterates over the number of epochs
step = 0
for i in range(params["num_epochs"]):

  IPython.display.clear_output(wait=True)
  print("Starting epoch {}/{}".format(i+1, params["num_epochs"]))
  p_lesions = get_training_sample_probabilities(all_imgs, dbvae)

  for j in tqdm(range(loader_ISIC.get_train_size() // params["batch_size"])):
    # load a batch of data
    (x, y) = loader_ISIC.get_batch(params["batch_size"], p_pos=p_lesions)

    # loss optimization
    loss = debiasing_train_step(x, y)
    experiment.log_metric("loss", loss.numpy().mean(), step=step, epoch=i+1)
    loss_history_2.append(loss.numpy().mean())
    # plot the progress every 200 steps
    if j % 500 == 0:
      mdl.util.plot_sample(x, y, dbvae)

    step += 1

experiment.end()

In [ ]:
### Training the DB-VAE on ISIC + DiDI ###
import IPython
# Hyperparameters
params = dict(
  batch_size = 32,
  learning_rate = 5e-4,
  latent_dim = 100,
  num_epochs = 50, #DB-VAE needs slightly more epochs to train
)

experiment = create_experiment("Model_4", params)

# instantiate a new DB-VAE model and optimizer
dbvae = DB_VAE(params["latent_dim"])
optimizer = tf.keras.optimizers.Adam(params["learning_rate"])

@tf.function
def debiasing_train_step(x, y):

  with tf.GradientTape() as tape:
    y_logit, z_mean, z_logsigma, x_recon = dbvae(x)
    loss, class_loss = debiasing_loss_function(x, x_recon, y, y_logit, z_mean, z_logsigma)
  grads = tape.gradient(loss, dbvae.trainable_variables)
  optimizer.apply_gradients(zip(grads, dbvae.trainable_variables))
  return loss

all_imgs = loader_ISIC_DiDI.get_all_train_faces()

if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists

loss_history_2 = mdl.util.LossHistory(smoothing_factor=0.99) # to record loss evolution
# The training loop -- outer loop iterates over the number of epochs
step = 0
for i in range(params["num_epochs"]):

  IPython.display.clear_output(wait=True)
  print("Starting epoch {}/{}".format(i+1, params["num_epochs"]))
  p_lesions = get_training_sample_probabilities(all_imgs, dbvae)

  for j in tqdm(range(loader_ISIC_DiDI.get_train_size() // params["batch_size"])):
    # load a batch of data
    (x, y) = loader_ISIC_DiDI.get_batch(params["batch_size"], p_pos=p_lesions)

    # loss optimization
    loss = debiasing_train_step(x, y)
    experiment.log_metric("loss", loss.numpy().mean(), step=step, epoch=i+1)
    loss_history_2.append(loss.numpy().mean())
    # plot the progress every 200 steps
    if j % 500 == 0:
      mdl.util.plot_sample(x, y, dbvae)

    step += 1

experiment.end()

In [ ]:
### Training the DB-VAE on ISIC + ArGI ###
import IPython
# Hyperparameters
params = dict(
  batch_size = 32,
  learning_rate = 5e-4,
  latent_dim = 100,
  num_epochs = 50, #DB-VAE needs slightly more epochs to train
)

experiment = create_experiment("Model_4", params)

# instantiate a new DB-VAE model and optimizer
dbvae = DB_VAE(params["latent_dim"])
optimizer = tf.keras.optimizers.Adam(params["learning_rate"])

@tf.function
def debiasing_train_step(x, y):

  with tf.GradientTape() as tape:
    y_logit, z_mean, z_logsigma, x_recon = dbvae(x)
    loss, class_loss = debiasing_loss_function(x, x_recon, y, y_logit, z_mean, z_logsigma)
  grads = tape.gradient(loss, dbvae.trainable_variables)
  optimizer.apply_gradients(zip(grads, dbvae.trainable_variables))
  return loss

all_imgs = loader_ISIC_ArGI.get_all_train_faces()

if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists

loss_history_2 = mdl.util.LossHistory(smoothing_factor=0.99) # to record loss evolution
# The training loop -- outer loop iterates over the number of epochs
step = 0
for i in range(params["num_epochs"]):

  IPython.display.clear_output(wait=True)
  print("Starting epoch {}/{}".format(i+1, params["num_epochs"]))
  p_lesions = get_training_sample_probabilities(all_imgs, dbvae)

  for j in tqdm(range(loader_ISIC_ArGI.get_train_size() // params["batch_size"])):
    # load a batch of data
    (x, y) = loader_ISIC_ArGI.get_batch(params["batch_size"], p_pos=p_lesions)

    # loss optimization
    loss = debiasing_train_step(x, y)
    experiment.log_metric("loss", loss.numpy().mean(), step=step, epoch=i+1)
    loss_history_2.append(loss.numpy().mean())
    # plot the progress every 200 steps
    if j % 500 == 0:
      mdl.util.plot_sample(x, y, dbvae)

    step += 1

experiment.end()

In [ ]:
### Obtain Loss Values Over Epoch ###
steps = len(loss_history_2.get())
print(steps)

epochs = np.uint8(params['num_epochs'])
batches_per_epoch = np.uint8(steps/epochs)

loss_hist = np.zeros((steps,1))
for i in range(steps):
    loss_hist[i] = loss_history_2.get()[i]
loss_hist = loss_hist.reshape(epochs, batches_per_epoch)

row_means = np.zeros((epochs))
for i in range(epochs):
    row_means[i] = loss_hist.sum(axis=1)[i]

row_means = row_means/batches_per_epoch
for mean in row_means:
    print(mean)

In [ ]:
### Evaluation of standard CNN on ISIC ###
n=30
accuracies = np.zeros((n,1))

for i in range(n):
    (batch_x, batch_y) = loader_ISIC.get_batch(256) # Matches batch size
    y_pred_standard = tf.round(tf.nn.sigmoid(dbvae.predict(batch_x)))
    acc_standard = tf.reduce_mean(tf.cast(tf.equal(batch_y, y_pred_standard), tf.float32))
    accuracies[i] = acc_standard.numpy()

print(accuracies.mean())
print(accuracies.std())

In [ ]:
### Evaluation of standard CNN on ISIC_DiDI ###
n=30
accuracies = np.zeros((n,1))

for i in range(n):
    (batch_x, batch_y) = loader_ISIC_DiDI.get_batch(256) # Matches batch size
    y_pred_standard = tf.round(tf.nn.sigmoid(dbvae.predict(batch_x)))
    acc_standard = tf.reduce_mean(tf.cast(tf.equal(batch_y, y_pred_standard), tf.float32))
    accuracies[i] = acc_standard.numpy()

print(accuracies.mean())
print(accuracies.std())